In [1]:
from ISLP import load_data
import pandas as pd
import sklearn.model_selection as skm
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as skl
from sklearn.pipeline import Pipeline

In [2]:
df = load_data("College")
df

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,Yes,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


In [3]:
# Drop private var since it's categorical (won't work well for ridge and lasso)
# Dropping private and apps
X = df.iloc[:,3:]
X

,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


In [4]:
y = df['Apps']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [5]:
# Ordinary Least squares
# Adj Rsquared is 0.852, multiple variables have a high p-val
results = sm.OLS(y_train,X_train).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   Apps   R-squared (uncentered):                   0.856
Model:                            OLS   Adj. R-squared (uncentered):              0.852
Method:                 Least Squares   F-statistic:                              224.0
Date:                Wed, 05 Jun 2024   Prob (F-statistic):                   9.87e-227
Time:                        12:31:41   Log-Likelihood:                         -5231.1
No. Observations:                 582   AIC:                                  1.049e+04
Df Residuals:                     567   BIC:                                  1.056e+04
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Enroll          2.8263      0.340      8.317      0.000       2.159       3.494
Top10perc      28.6049     11.569      2.473      0.014       5.881      51.328
Top25perc      -2.4184      9.416     -0.257      0.797     -20.913      16.076
F.Undergrad     0.1464      0.069      2.125      0.034       0.011       0.282
P.Undergrad    -0.0025      0.065     -0.038      0.970      -0.130       0.125
Outstate        0.0083      0.038      0.215      0.830      -0.067       0.084
Room.Board      0.3495      0.103      3.407      0.001       0.148       0.551
Books          -0.4362      0.479     -0.910      0.363      -1.377       0.505
Personal       -0.3451      0.126     -2.741      0.006      -0.592      -0.098
PhD            -0.5654     10.027     -0.056      0.955     -20.261      19.130
Terminal      -16.6496     10.523     -1.582      0.114     -37.319       4.020
S.F.Ratio     -31.4568     21.457     -1.466      0.143     -73.601      10.688
perc.alumni   -33.8881      8.953     -3.785      0.000     -51.473     -16.303
Expend          0.0619      0.025      2.438      0.015       0.012       0.112
Grad.Rate      11.2932      6.217      1.816      0.070      -0.918      23.505
==============================================================================
Omnibus:                      862.941   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           406538.104
Skew:                           7.813   Prob(JB):                         0.00
Kurtosis:                     131.531   Cond. No.                     4.46e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 4.46e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
pred = results.predict(X_test)
score = ((y_test - pred)**2).mean()
score

2051258.5777322883

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV, LassoCV

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
rcv = RidgeCV(alphas=np.linspace(0.1, 100, 1000), cv=10)
rcv.fit(X_train_scaled, y_train)

RidgeCV(alphas=array([  0.1,   0.2,   0.3,   0.4,   0.5,   0.6,   0.7,   0.8,   0.9,
         1. ,   1.1,   1.2,   1.3,   1.4,   1.5,   1.6,   1.7,   1.8,
         1.9,   2. ,   2.1,   2.2,   2.3,   2.4,   2.5,   2.6,   2.7,
         2.8,   2.9,   3. ,   3.1,   3.2,   3.3,   3.4,   3.5,   3.6,
         3.7,   3.8,   3.9,   4. ,   4.1,   4.2,   4.3,   4.4,   4.5,
         4.6,   4.7,   4.8,   4.9,   5. ,   5.1,   5.2,   5.3,   5.4,
         5.5,   5.6,   5.7,   5.8,   5.9,   6. ,   6.1,   6.2,   6.3,
         6.4,   6.5,   6.6,   6.7,   6.8,   6.9,   7. ,   7.1,   7.2,
         7.3,   7.4,   7.5,   7.6,   7.7,   7.8,   7.9,   8. ,   8.1,
         8.2,   8.3,   8.4,...
        92.8,  92.9,  93. ,  93.1,  93.2,  93.3,  93.4,  93.5,  93.6,
        93.7,  93.8,  93.9,  94. ,  94.1,  94.2,  94.3,  94.4,  94.5,
        94.6,  94.7,  94.8,  94.9,  95. ,  95.1,  95.2,  95.3,  95.4,
        95.5,  95.6,  95.7,  95.8,  95.9,  96. ,  96.1,  96.2,  96.3,
        96.4,  96.5,  96.6,  96.7,  96.8,  96.9,  97. ,  97.1,  97.2,
        97.3,  97.4,  97.5,  97.6,  97.7,  97.8,  97.9,  98. ,  98.1,
        98.2,  98.3,  98.4,  98.5,  98.6,  98.7,  98.8,  98.9,  99. ,
        99.1,  99.2,  99.3,  99.4,  99.5,  99.6,  99.7,  99.8,  99.9,
       100. ]),
        cv=10)

In [12]:
rcv.score(X_test_scaled, y_test)

0.8016260052257205

In [17]:
((rcv.predict(X_test_scaled)-y_test)**2).mean()
# MSE For ridge isn't much better than for OLS

2033100.6020789458

In [19]:
lcv = LassoCV(alphas=np.linspace(0.1,100,1000), cv=10)
lcv.fit(X_train_scaled, y_train)
lcv.score(X_test_scaled, y_test)

0.8085798654746663

In [21]:
((lcv.predict(X_test_scaled)-y_test)**2).mean()
# MSE for lasso is a little better than OLS and ridge

1961831.6967217114

In [23]:
lcv.coef_

array([2757.53402951,  289.89144306,   56.3913505 ,  535.10303993,
         -0.        ,    0.        ,  444.04650551,    0.        ,
        -52.39735162,   -0.        ,   -0.        ,   62.49785743,
       -280.56096451,  352.17382641,  282.1391372 ])

In [26]:
from sklearn.decomposition import PCA
# PCR
pca = PCA(n_components=2)
linreg = skl.LinearRegression()
pipe = Pipeline([('pca',pca),
                 ('linreg', linreg)])
pipe.fit(X,y)
pipe.named_steps['linreg'].coef_

array([0.04797823, 0.64631343])

In [28]:
# Standardizing the data first
pipe = Pipeline([('scaler',scaler),
                 ('pca',pca),
                 ('linreg', linreg)])
pipe.fit(X,y)
pipe.named_steps['linreg'].coef_

array([ 428.74164613, 1632.26879212])

In [56]:
K = 30
kfold = skm.KFold(K,
                  random_state=0,
                  shuffle=True)
# Fixing params to vary the components
param_grid = {'pca__n_components': range(1,15)}
grid = skm.GridSearchCV(pipe,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
grid.fit(X,y)

GridSearchCV(cv=KFold(n_splits=30, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA(n_components=2)),
                                       ('linreg', LinearRegression())]),
             param_grid={'pca__n_components': range(1, 15)},
             scoring='neg_mean_squared_error')

In [57]:
(-grid.cv_results_['mean_test_score']).min()
# This was the worst by far, maybe I did something wrong ?

3442731.897412277

In [62]:
np.argmin(-grid.cv_results_['mean_test_score'])
# PCR did best with 9 predictors

9

In [65]:
from sklearn.cross_decomposition import PLSRegression
# Partial Least Squares
pls = PLSRegression(n_components=2,
                    scale=True)
pls.fit(X,y)

PLSRegression()

In [67]:
# Use CV to choose num of components
param_grid = {'n_components': range(1,20)}
grid = skm.GridSearchCV(pls,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
grid.fit(X,y)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
120 fits failed out of a total of 570.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py", line 640, in fit
    super().fit(X, Y)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python

GridSearchCV(cv=KFold(n_splits=30, random_state=0, shuffle=True),
             estimator=PLSRegression(),
             param_grid={'n_components': range(1, 20)},
             scoring='neg_mean_squared_error')